<a href="https://colab.research.google.com/github/Isafon/ECE528/blob/main/ECE528_ASN2_Q1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q1 Part B ECE528 LA2 - Isa Fontana

In [2]:
!pip -q install -U "autokeras>=2.0.0b1" "keras-tuner>=1.4.7"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 14.5 MB/s eta 0:00:00


In [3]:
# I give up doing all the fancy labeling, it's taking too long,
# see write up for detailed comments if needed
import os, random, numpy as np
import tensorflow as tf
from tensorflow import keras
import autokeras as ak

SEED = 0
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

print("TF:", tf.__version__, "| Keras:", keras.__version__, "| AutoKeras:", ak.__version__)
tf.config.experimental.list_physical_devices('GPU')

TF: 2.19.0 | Keras: 3.10.0 | AutoKeras: 2.0.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

val_size = 10000
x_val, y_val = x_train[-val_size:], y_train[-val_size:]
x_train, y_train = x_train[:-val_size], y_train[:-val_size]

# scale to [0,1] and add channel dimension
x_train = (x_train.astype("float32")/255.0)[..., None]
x_val   = (x_val.astype("float32")/255.0)[..., None]
x_test  = (x_test.astype("float32")/255.0)[..., None]

x_train.shape, x_val.shape, x_test.shape

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


((50000, 28, 28, 1), (10000, 28, 28, 1), (10000, 28, 28, 1))

In [5]:
def build_baseline():
    model = keras.Sequential([
        keras.layers.Input((28,28,1)),
        keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
        keras.layers.MaxPooling2D(),
        keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
        keras.layers.MaxPooling2D(),
        keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
        keras.layers.MaxPooling2D(),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.30),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax"),
    ], name="fmnist_cnn_baseline")
    model.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

model_q1a = build_baseline()

ckpt_dir = "checkpoints_q1a"; os.makedirs(ckpt_dir, exist_ok=True)
ckpt = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(ckpt_dir, "best_q1a.keras"),
    monitor="val_accuracy", mode="max", save_best_only=True)

callbacks_q1a = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=2, min_lr=1e-5, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1),
    ckpt
]

history_a = model_q1a.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=15, batch_size=128,
    callbacks=callbacks_q1a, verbose=2
)
loss_a, acc_a = model_q1a.evaluate(x_test, y_test, verbose=0)
print(f"Hand-designed test accuracy: {acc_a:.4f}")

Epoch 1/15
391/391 - 15s - 38ms/step - accuracy: 0.7837 - loss: 0.5917 - val_accuracy: 0.8659 - val_loss: 0.3816 - learning_rate: 1.0000e-03
Epoch 2/15
391/391 - 1s - 3ms/step - accuracy: 0.8689 - loss: 0.3581 - val_accuracy: 0.8892 - val_loss: 0.3090 - learning_rate: 1.0000e-03
Epoch 3/15
391/391 - 1s - 3ms/step - accuracy: 0.8893 - loss: 0.3037 - val_accuracy: 0.8966 - val_loss: 0.2843 - learning_rate: 1.0000e-03
Epoch 4/15
391/391 - 1s - 3ms/step - accuracy: 0.8995 - loss: 0.2728 - val_accuracy: 0.9014 - val_loss: 0.2727 - learning_rate: 1.0000e-03
Epoch 5/15
391/391 - 1s - 3ms/step - accuracy: 0.9109 - loss: 0.2461 - val_accuracy: 0.9039 - val_loss: 0.2582 - learning_rate: 1.0000e-03
Epoch 6/15
391/391 - 1s - 3ms/step - accuracy: 0.9156 - loss: 0.2298 - val_accuracy: 0.9097 - val_loss: 0.2453 - learning_rate: 1.0000e-03
Epoch 7/15
391/391 - 1s - 3ms/step - accuracy: 0.9206 - loss: 0.2143 - val_accuracy: 0.9098 - val_loss: 0.2462 - learning_rate: 1.0000e-03
Epoch 8/15
391/391 - 1s -

In [6]:
import autokeras as ak
import keras

clf = ak.ImageClassifier(
    max_trials=2,      # per assignment
    overwrite=True,
    seed=SEED
)

# Optional checkpoint while AK trains (saves the raw Keras model during trials)
ckpt_dir_ak = "checkpoints_ak"; os.makedirs(ckpt_dir_ak, exist_ok=True)
ak_ckpt = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(ckpt_dir_ak, "best_autokeras.keras"),
    monitor="val_accuracy", mode="max", save_best_only=True)

clf.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=20, batch_size=128, verbose=2,
    callbacks=[ak_ckpt]
)

Trial 2 Complete [03h 00m 33s]
val_loss: 0.2288699746131897

Best val_loss So Far: 0.20673197507858276
Total elapsed time: 03h 01m 17s
Epoch 1/20
391/391 - 8s - 21ms/step - accuracy: 0.8198 - loss: 0.5018 - val_accuracy: 0.8801 - val_loss: 0.3390
Epoch 2/20
391/391 - 2s - 4ms/step - accuracy: 0.8766 - loss: 0.3470 - val_accuracy: 0.8976 - val_loss: 0.2905
Epoch 3/20
391/391 - 2s - 4ms/step - accuracy: 0.8905 - loss: 0.3069 - val_accuracy: 0.9031 - val_loss: 0.2695
Epoch 4/20
391/391 - 2s - 4ms/step - accuracy: 0.8988 - loss: 0.2827 - val_accuracy: 0.9085 - val_loss: 0.2540
Epoch 5/20
391/391 - 2s - 4ms/step - accuracy: 0.9048 - loss: 0.2618 - val_accuracy: 0.9133 - val_loss: 0.2428
Epoch 6/20
391/391 - 2s - 4ms/step - accuracy: 0.9116 - loss: 0.2492 - val_accuracy: 0.9144 - val_loss: 0.2402
Epoch 7/20
391/391 - 2s - 4ms/step - accuracy: 0.9131 - loss: 0.2380 - val_accuracy: 0.9159 - val_loss: 0.2355
Epoch 8/20
391/391 - 2s - 4ms/step - accuracy: 0.9181 - loss: 0.2278 - val_accuracy: 0.

In [7]:
best = clf.export_model()
best.compile(optimizer="adam",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])
loss_b, acc_b = best.evaluate(x_test, y_test, verbose=0)
print(f"AutoKeras best test accuracy: {acc_b:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


AutoKeras best test accuracy: 0.9206


In [8]:
model_q1a.save("q1a_handdesigned.keras")
best.save("q1b_autokeras_best.keras")

keras.utils.plot_model(model_q1a, to_file="q1a_model.png", show_shapes=True, dpi=120)
keras.utils.plot_model(best,       to_file="q1b_autokeras_model.png", show_shapes=True, dpi=120)

print("Saved: q1a_handdesigned.keras, q1b_autokeras_best.keras, q1a_model.png, q1b_autokeras_model.png")

Saved: q1a_handdesigned.keras, q1b_autokeras_best.keras, q1a_model.png, q1b_autokeras_model.png


In [9]:
import pandas as pd
pd.DataFrame({
    "Model": ["Hand-designed (Q1a)", "AutoKeras (best)"],
    "Test Accuracy": [acc_a, acc_b]
})

,Model,Test Accuracy
0,Hand-designed (Q1a),0.9237
1,AutoKeras (best),0.9206
